# Replication study

Replicate the DeepAR results for:

- Rangapuram, Syama Sundar, et al. "Deep state space models for time series forecasting." Advances in Neural Information Processing Systems. 2018.

## Data

Among the data there are two publicly available datasets, namely `electricity` and `traffic`

- `electricity` - Hourly time series of electricity consumption of 370 customers
- `traffic` - Hourly occupancy rates (between 0 and 1) of 963 car lanes of San Francisco bay area freeways

Both datasets exhibit hourly and daily seasonal patterns.

### Benchmark Methods 

- `auto.arima` - automatic ARIMA framework from R *forecast* package
- `ets` - automatic expoential smoothing, also from R *forecast* package

Benchmark methods are compared to `DeepAR` (this notebook) and `DeepState` (next notebook). For the paper, the built-in algorithm of the AWS SageMaker is used. I will see if the `gluonts.model.deepar` implementation is comparable.  

### RNN-based methods 

Inputs for `DeepAR`and `DeepState`: 

- Independent feature representing the category (i.e. the index)
- time-based features like hour-of-the-day, day-of-the-week, day-of-the-month
- `DeepState` - Size of state space model directly depends on the granularity of the time series which determines the number of seasons. 
 - For hourly data - hour-of-the day (24 seasons), day-of-the-week (7 seasons) models and hence latent dimension is 31
- Train each method on all time series of these datasets but vary the size of the training range $T_i \in {14, 21, 28}$ days.
- Evaluate on the next $\tau = 7$ days using *p50* and *p90* quantile loss.

#### Imports

In [1]:
# standard imports
# import numpy as np
# import pandas as pd

# json
# import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

# seeds
mx.random.seed(42)
# np.random.seed(42)


INFO:root:Using CPU


In [35]:
dataset = get_dataset("electricity")

# Inspect data
print(len(list(dataset.train)))
print(len(list(dataset.test)))

entry = list(dataset.test)[0]

print("entry:\n", entry)
print("metadata: \n", dataset.metadata)
print("prediction_length:n", dataset.metadata.prediction_length)

321
2247
entry:
 {'start': Timestamp('2012-01-01 00:00:00', freq='H'), 'target': array([14., 18., 21., ...,  8.,  8.,  6.], dtype=float32), 'feat_static_cat': array([0]), 'source': SourceContext(source=Span(path=WindowsPath('C:/Users/TM/.mxnet/gluon-ts/datasets/electricity/test/data.json'), line=1), row=1)}
metadata: 
 MetaData freq='1H' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='321'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=24
prediction_length:n 24


In [18]:
mx.random.seed(42)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,      # default: 100
        batch_size=32,      # default: 32
        num_batches_per_epoch=50,      #default: 50
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=14,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=7,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 19863
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:15<00:00,  3.31it/s, avg_epoch_loss=6.44]
INFO:root:Epoch[0] Elapsed time 15.100 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.437449
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:19<00:00,  2.53it/s, avg_epoch_loss=5.88]
INFO:root:Epoch[1] Elapsed time 19.789 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=5.880868
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:16<00:00,  3.10it/s, avg_epoch_loss=5.78]
INFO:root:Epoch[2] Elapsed time 16.392 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=5.779489
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:15<00:00,  3.23it/s, avg_epoch_loss=5.62]
INFO:root:Epoch[3] Elapsed time 15.471 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=5.622680
INFO:root:Epoch[4] Learning rate is 0.

100%|██████████| 50/50 [00:14<00:00,  3.39it/s, avg_epoch_loss=5.12]
INFO:root:Epoch[73] Elapsed time 14.736 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.115044
INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 50/50 [00:14<00:00,  3.47it/s, avg_epoch_loss=5.03]
INFO:root:Epoch[74] Elapsed time 14.416 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.025519
INFO:root:Epoch[75] Learning rate is 0.00025
100%|██████████| 50/50 [00:13<00:00,  3.63it/s, avg_epoch_loss=5.13]
INFO:root:Epoch[75] Elapsed time 13.775 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.128985
INFO:root:Epoch[76] Learning rate is 0.00025
100%|██████████| 50/50 [00:14<00:00,  3.40it/s, avg_epoch_loss=5.13]
INFO:root:Epoch[76] Elapsed time 14.725 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.128015
INFO:root:Epoch[77] Learning rate is 0.00025
100%|██████████| 50/50 [00:13<00:00,  3.60it/s, avg_epoch_loss=5.09]
INFO:root:Epoch[77] Elapsed time 13.880 sec

{'Coverage[0.1]': 0.06948947803420413,
 'Coverage[0.2]': 0.14164918303770155,
 'Coverage[0.3]': 0.21787780532774015,
 'Coverage[0.4]': 0.304405874499333,
 'Coverage[0.5]': 0.3965922817725233,
 'Coverage[0.6]': 0.48451904126136386,
 'Coverage[0.7]': 0.5940619238349548,
 'Coverage[0.8]': 0.7050034967257958,
 'Coverage[0.9]': 0.8215398308856311,
 'MAE_Coverage': 0.08498456495786139,
 'MASE': 0.8305677374334318,
 'MSE': 5217725.340263367,
 'MSIS': 10.504062832445506,
 'ND': 0.07373828362188806,
 'NRMSE': 0.7078700875494279,
 'QuantileLoss[0.1]': 1436492.9250928767,
 'QuantileLoss[0.2]': 2356537.727963176,
 'QuantileLoss[0.3]': 3017019.4001543047,
 'QuantileLoss[0.4]': 3463336.3744392763,
 'QuantileLoss[0.5]': 3742666.9418717753,
 'QuantileLoss[0.6]': 3875761.2026613504,
 'QuantileLoss[0.7]': 3789808.0337280156,
 'QuantileLoss[0.8]': 3463316.226479943,
 'QuantileLoss[0.9]': 2688687.940038397,
 'RMSE': 2284.2340817576833,
 'abs_error': 3742666.9599580765,
 'abs_target_mean': 3226.91143747218

In [36]:
dataset = get_dataset("traffic")

# Inspect data
print(len(list(dataset.train)))
print(len(list(dataset.test)))

entry = list(dataset.test)[0]

print("entry:\n", entry)
print("metadata: \n", dataset.metadata)
print("prediction_length:n", dataset.metadata.prediction_length)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\traffic.


862
6034
entry:
 {'start': Timestamp('2015-01-01 00:00:00', freq='H'), 'target': array([0.0048, 0.0072, 0.004 , ..., 0.0467, 0.0412, 0.0386], dtype=float32), 'feat_static_cat': array([0]), 'source': SourceContext(source=Span(path=WindowsPath('C:/Users/TM/.mxnet/gluon-ts/datasets/traffic/test/data.json'), line=1), row=1)}
metadata: 
 MetaData freq='H' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='862'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=24
prediction_length:n 24


In [37]:
mx.random.seed(42)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=1,      # default: 100
        batch_size=32,      # default: 32
        num_batches_per_epoch=50,      #default: 50
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=7,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 30683
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:12<00:00,  3.87it/s, avg_epoch_loss=-2.27]
INFO:root:Epoch[0] Elapsed time 12.995 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-2.272214
INFO:root:Loading parameters from best epoch (0)
INFO:root:Final loss: -2.2722144603729246 (occurred at epoch 0)
INFO:root:End model training
Running evaluation: 100%|██████████| 6034/6034 [07:43<00:00, 13.01it/s]


{'Coverage[0.1]': 0.04007055258298286,
 'Coverage[0.2]': 0.09851318717742238,
 'Coverage[0.3]': 0.1728656659879706,
 'Coverage[0.4]': 0.26177849329987035,
 'Coverage[0.5]': 0.36644727496567053,
 'Coverage[0.6]': 0.4704768218192145,
 'Coverage[0.7]': 0.590143946209577,
 'Coverage[0.8]': 0.7080472560253835,
 'Coverage[0.9]': 0.8293598181732146,
 'MAE_Coverage': 0.1069218870842993,
 'MASE': 1.079393405723361,
 'MSE': 0.0013130994038727365,
 'MSIS': 17.915881401398057,
 'ND': 0.20442417919242933,
 'NRMSE': 0.4393772762501815,
 'QuantileLoss[0.1]': 577.6576995693454,
 'QuantileLoss[0.2]': 892.3911496231565,
 'QuantileLoss[0.3]': 1126.6327669337725,
 'QuantileLoss[0.4]': 1301.3987775167056,
 'QuantileLoss[0.5]': 1424.2191273349454,
 'QuantileLoss[0.6]': 1499.1768028632039,
 'QuantileLoss[0.7]': 1505.014514826471,
 'QuantileLoss[0.8]': 1420.4288216932678,
 'QuantileLoss[0.9]': 1177.794573688228,
 'RMSE': 0.03623671348056742,
 'abs_error': 1424.2191269584,
 'abs_target_mean': 0.082472889335164

In [22]:
# import rpy2
# print(rpy2.__version__)

# # importing packages
# from rpy2.robjects.packages import importr
# import rpy2.robjects.packages as rpackages

# base=importr("base")
# utils=importr("utils")

# utils = rpackages.importr("utils")
# utils.chooseCRANmirror(ind=1)

# utils.install_packages("forecast")
# utils.install_packages("nnfor")


In [ ]:
print("Done!")